# 1. Silver Layer: Create totals and followers views

In [0]:
try:
    import sys
    import os

    sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '../utils')))

    from pipeline_utils import get_valid_parameter
except:
    import re

    def get_valid_parameter(parameter_key: str):
        """
        Get a valid parameter value from the Databricks widgets.
        Hardened to thwart SQL injection attacks.
        """
        parameter_value = dbutils.widgets.get(parameter_key)
        
        # Parameter_value must be a string with only alphanumeric characters and underscores
        if not re.fullmatch(r'[a-zA-Z0-9_]+', parameter_value):
            raise ValueError(f"Invalid parameter value for {parameter_key}: {parameter_value}")
        
        # Disallow dangerous SQL keywords and patterns
        forbidden_patterns = [
            r'--', r';', r"'", r'"', r'/\*', r'\*/', r'xp_', r'char\(', r'nchar\(', r'varchar\(', r'\balter\b', r'\bdrop\b', r'\binsert\b', r'\bdelete\b', r'\bupdate\b', r'\bselect\b', r'\bcreate\b', r'\bexec\b', r'\bunion\b', r'\bor\b', r'\band\b'
        ]
        for pattern in forbidden_patterns:
            if re.search(pattern, parameter_value, re.IGNORECASE):
                raise ValueError(f"Potentially dangerous value for {parameter_key}: {parameter_value} (pattern matched: {pattern})")
        return parameter_value

In [0]:
# 1. define our constants
try:
    BRONZE_CATALOG = get_valid_parameter("BRONZE_CATALOG")
    BRONZE_SCHEMA = get_valid_parameter("BRONZE_SCHEMA")
    BRONZE_TOTALS_TABLE = get_valid_parameter("BRONZE_TOTALS_TABLE")
    BRONZE_FOLLOWERS_TABLE = get_valid_parameter("BRONZE_FOLLOWERS_TABLE")

    SILVER_CATALOG = get_valid_parameter("SILVER_CATALOG")
    SILVER_SCHEMA = get_valid_parameter("SILVER_SCHEMA")
    SILVER_TOTALS_TABLE = get_valid_parameter("SILVER_TOTALS_TABLE")
    SILVER_FOLLOWERS_TABLE = get_valid_parameter("SILVER_FOLLOWERS_TABLE")

    print("Loaded all widget values")
except:
    BRONZE_CATALOG = "bronze"
    BRONZE_SCHEMA = "linkedin"
    BRONZE_TOTALS_TABLE = "totals"
    BRONZE_FOLLOWERS_TABLE = "followers"

    SILVER_CATALOG = "silver"
    SILVER_SCHEMA = "linkedin"
    SILVER_TOTALS_TABLE = "totals"
    SILVER_FOLLOWERS_TABLE = "followers"

    print("Failed to load widget values, using default values")


In [0]:
# Create Silver Totals View
spark.sql(f"""
CREATE OR REPLACE VIEW {SILVER_CATALOG}.{SILVER_SCHEMA}.{SILVER_TOTALS_TABLE} AS
SELECT date, impressions, engagements
FROM {BRONZE_CATALOG}.{BRONZE_SCHEMA}.{BRONZE_TOTALS_TABLE}
""")

# Create Silver Followers View
spark.sql(f"""
CREATE OR REPLACE VIEW {SILVER_CATALOG}.{SILVER_SCHEMA}.{SILVER_FOLLOWERS_TABLE} AS
SELECT date, new_followers
FROM {BRONZE_CATALOG}.{BRONZE_SCHEMA}.{BRONZE_FOLLOWERS_TABLE}
""")